In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
# import seaborn as sns
# from pathlib import Path
import pandas_datareader as web

In [2]:
# S&P 500
sp_500 = web.get_data_yahoo("^GSPC", start="2017-10-31", end="2022-10-31")
sp_500

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-10-31,2578.290039,2572.149902,2575.989990,2575.260010,3827230000,2575.260010
2017-11-01,2588.399902,2574.919922,2583.209961,2579.360107,3813180000,2579.360107
2017-11-02,2581.110107,2566.169922,2579.459961,2579.850098,4048270000,2579.850098
2017-11-03,2588.419922,2576.770020,2581.929932,2587.840088,3567710000,2587.840088
2017-11-06,2593.379883,2585.659912,2587.469971,2591.129883,3539080000,2591.129883
...,...,...,...,...,...,...
2022-10-25,3862.850098,3799.439941,3799.439941,3859.110107,4843120000,3859.110107
2022-10-26,3886.149902,3824.070068,3825.969971,3830.600098,4817310000,3830.600098
2022-10-27,3859.949951,3803.790039,3834.689941,3807.300049,4687320000,3807.300049


In [3]:
# Oil
oil_data = web.get_data_yahoo("CL=F", start="2017-10-31", end="2022-10-31")

oil_data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-10-31,54.849998,53.930000,54.080002,54.380001,497295.0,54.380001
2017-11-01,55.220001,53.889999,54.650002,54.299999,671038.0,54.299999
2017-11-02,54.840000,53.990002,54.279999,54.540001,482846.0,54.540001
2017-11-03,55.759998,54.400002,54.730000,55.639999,614159.0,55.639999
2017-11-06,57.610001,55.660000,55.970001,57.349998,861713.0,57.349998
...,...,...,...,...,...,...
2022-10-26,88.410004,84.139999,85.070000,87.910004,287182.0,87.910004
2022-10-27,89.790001,87.330002,88.260002,89.080002,292556.0,89.080002
2022-10-28,88.760002,87.080002,88.669998,87.900002,263404.0,87.900002


In [4]:
# BTC
btc_data2 = web.get_data_yahoo("BTC-USD", start="2017-10-31", end="2022-10-31")
btc_data2

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-10-31,6470.430176,6103.330078,6132.020020,6468.399902,2311379968,6468.399902
2017-11-01,6767.310059,6377.879883,6440.970215,6767.310059,2870320128,6767.310059
2017-11-02,7367.330078,6758.720215,6777.770020,7078.500000,4653770240,7078.500000
2017-11-03,7461.290039,7002.939941,7087.529785,7207.759766,3369860096,7207.759766
2017-11-04,7492.859863,7031.279785,7164.479980,7379.950195,2483800064,7379.950195
...,...,...,...,...,...,...
2022-10-28,20724.980469,20086.068359,20287.957031,20595.351562,43994715910,20595.351562
2022-10-29,20988.394531,20566.484375,20595.103516,20818.476562,40369840645,20818.476562
2022-10-30,20917.005859,20547.462891,20817.982422,20635.603516,31486345556,20635.603516
